In [8]:
import os
import re
import time
import json
import openai
import numpy as np
import traceback
import matplotlib.pyplot as plt
from dotenv import load_dotenv, find_dotenv

import langchain.chat_models
import langchain.schema
from langchain.callbacks import get_openai_callback
from langchain import OpenAI
from langchain.schema import (HumanMessage, AIMessage)

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

_ = load_dotenv(find_dotenv())
open_api_key = os.environ["OPENAI_API_KEY"]

from openai_textgenerator import TextGenerator
from summarizer import Summarizer
from datamodel import Message, LLMResponse

# Test TextGenerator Class

In [ ]:
config =  {'temperature' : 0, 'max_tokens': 300, 'n': 3}
chatbot = TextGenerator()
prompt = "It seems like today is cold in bay area time to get cozy in the house. what do you think?"
messages = [
    {
        "role": "system",
        "content": "You are a sassy funny assistant tasked to respond in a sarcastic tone",
    },
    {
        "role": "user",
        "content": f"{prompt}"
    }

]
res = chatbot.generate(messages=messages, config=config)

In [ ]:
res.text

# Test Summerizer Class

In [5]:
text_gen_config = {'max_tokens':1000}
text_gen = TextGenerator()
df = pd.read_excel("../data/ROBERT_KING.xlsx", index_col=0)
get_summary = Summarizer()

In [3]:
summary_default = get_summary.summarize(data=df, n_samples=3, text_gen=text_gen, textgen_config=text_gen_config, summary_method="default")

In [4]:
summary_default

{'name': '',
 'file_name': '',
 'dataset_description': '',
 'fields': [{'column': 'Date',
   'properties': {'dtype': 'date',
    'min': '2023-08-01',
    'max': '2023-11-01',
    'samples': ['2023-09-10', '2023-08-23', '2023-09-25'],
    'num_unique_values': 93,
    'semantic_type': '',
    'description': ''}},
  {'column': 'Account_Number',
   'properties': {'dtype': 'number',
    'std': 0,
    'min': 1012345678,
    'max': 1012345679,
    'samples': [1012345679, 1012345678],
    'num_unique_values': 2,
    'semantic_type': '',
    'description': ''}},
  {'column': 'Transaction_Type',
   'properties': {'dtype': 'category',
    'samples': ['ACH', 'Transfer'],
    'num_unique_values': 6,
    'semantic_type': '',
    'description': ''}},
  {'column': 'Transaction_Purpose',
   'properties': {'dtype': 'category',
    'samples': ['Bills & utilities', 'Investment'],
    'num_unique_values': 16,
    'semantic_type': '',
    'description': ''}},
  {'column': 'Transaction_Amount',
   'propertie

In [15]:
summary_llm = get_summary.summarize(data=df, n_samples=3, text_gen=text_gen, textgen_config=text_gen_config, summary_method="llm")

In [16]:
summary_llm

{'name': 'Bank Transactions',
 'file_name': '',
 'dataset_description': 'A dataset containing bank transactions',
 'fields': [{'column': 'Date',
   'properties': {'dtype': 'date',
    'min': '2023-08-01',
    'max': '2023-11-01',
    'samples': ['2023-09-10', '2023-08-23', '2023-09-25'],
    'num_unique_values': 93,
    'semantic_type': 'date',
    'description': 'The date of the transaction'}},
  {'column': 'Account_Number',
   'properties': {'dtype': 'number',
    'std': 0,
    'min': 1012345678,
    'max': 1012345679,
    'samples': [1012345679, 1012345678],
    'num_unique_values': 2,
    'semantic_type': 'account_number',
    'description': 'The account number associated with the transaction'}},
  {'column': 'Transaction_Type',
   'properties': {'dtype': 'category',
    'samples': ['ACH', 'Transfer'],
    'num_unique_values': 6,
    'semantic_type': 'transaction_type',
    'description': 'The type of transaction'}},
  {'column': 'Transaction_Purpose',
   'properties': {'dtype': 'c